<a href="https://colab.research.google.com/github/Ajou-SFTB/VF-MARKET/blob/main/ml/HR_VITON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Preprocessing

### Openpose

In [ ]:
!git clone https://github.com/bioroid17/HR-VITON-forked.git HR-VITON

In [ ]:
# !cp /content/drive/MyDrive/CIHP_PGN.zip /content/
# !cp /content/drive/MyDrive/detectron2.zip /content/
# CIHP_PGN
!gdown https://drive.google.com/uc?id=1ZNfYG4AchmAhwN7f856s97v5V6vv-V_I
# detectron2
!gdown https://drive.google.com/uc?id=1olLrYa8x3HpEYp8Us5mPPaSNV6eJJdGN

In [ ]:
!unzip CIHP_PGN.zip
!unzip detectron2.zip

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl

In [ ]:
# !cp /content/drive/MyDrive/Colab\ Notebooks/openpose-models.zip /content
!gdown https://drive.google.com/uc?id=1SN1d31LQPV45FScXzQL423UQR4-zO7k6

In [ ]:
!unzip /content/openpose-models.zip

In [8]:
!mv /content/models/pose/body_25/pose_iter_584000.caffemodel  /content/openpose/models/pose/body_25/pose_iter_584000.caffemodel
!mv /content/models/pose/coco/pose_iter_440000.caffemodel  /content/openpose/models/pose/coco/pose_iter_440000.caffemodel
!mv /content/models/pose/mpi/pose_iter_160000.caffemodel  /content/openpose/models/pose/mpi/pose_iter_160000.caffemodel
!mv /content/models/face/pose_iter_116000.caffemodel  /content/openpose/models/face/pose_iter_116000.caffemodel
!mv /content/models/hand/pose_iter_102000.caffemodel  /content/openpose/models/hand/pose_iter_102000.caffemodel
!mv /content/models/hand/pose_iter_120000.caffemodel  /content/openpose/models/hand/pose_iter_120000.caffemodel

In [ ]:
# build openpose
!cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

In [10]:
!mkdir HR-VITON/data
!mkdir HR-VITON/data/test

# for storing input image
!mkdir HR-VITON/data/test/image

# # copy official provided data to image_path, you may need to download and unzip it in advance
# !cp ./test/image/000* ./image_path/

# create directories for generated results of OpenPose
!mkdir HR-VITON/data/test/openpose_json
!mkdir HR-VITON/data/test/openpose_img

!mkdir HR-VITON/data/test/cloth

In [ ]:
!cp /content/drive/MyDrive/human.jpg /content/HR-VITON/data/test/image
!mv /content/HR-VITON/data/test/image/human.jpg /content/HR-VITON/data/test/image/00000.jpg
!cp /content/drive/MyDrive/shirt.jpg /content/HR-VITON/data/test/cloth
!mv /content/HR-VITON/data/test/cloth/shirt.jpg /content/HR-VITON/data/test/cloth/00000.jpg

In [12]:
# # go to openpose directory
# %cd openpose
# # run openpose.bin
# !./build/examples/openpose/openpose.bin --image_dir ../HR-VITON/data/test/image --disable_blending --display 0 --write_json ../HR-VITON/data/test/openpose_json --write_images ../HR-VITON/data/test/openpose_img --num_gpu 1 --num_gpu_start 0

### Human parse

In [14]:
# %%bash
# FILE_NAME='./CIHP_pgn.zip'
# FILE_ID='1Mqpse5Gen4V4403wFEpv3w3JAsWw2uhk'

# curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=$FILE_ID" > /dev/null
# CODE="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"
# curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=${CODE}&id=$FILE_ID" -o $FILE_NAME

In [15]:
# !unzip CIHP_pgn.zip

In [16]:
# !git clone https://github.com/Engineering-Course/CIHP_PGN.git
# %cd CIHP_PGN

In [17]:
# !mkdir -p ./checkpoint
# !mkdir -p ./datasets/images
# # You also need to download dataset provided or use your own images
# !mv ../CIHP_pgn ./checkpoint/CIHP_pgn
# !cp ../test/image/000* ./datasets/images

In [18]:
!sudo apt-get install python3-pip
!python -m pip install --upgrade pip
!pip install matplotlib opencv-python==4.2.0.32 Pillow scipy tensorflow==1.15
!pip install ipykernel

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python3-pip python3-setuptools python3-wheel
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,677 kB of archives.
After this operation, 8,967 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-setuptools all 59.6.0-1.2ubuntu0.22.04.1 [339 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-wheel all 0.37.1-2ubuntu0.22.04.1 [32.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip all 22.0.2+dfsg-1ubuntu0.4 [1,305 kB]
Fetched 1,677 kB in 1s (1,988 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog

In [19]:
# !tf_upgrade_v2 --intree . --outtree . --copyotherfiles False

In [20]:
!mkdir /content/HR-VITON/data/test/image-parse-v3

In [21]:
# %cd CIHP_PGN
# !python ./inf_pgn.py --directory ../HR-VITON/data/test/image --output ../human_parse_output

In [23]:
# !cp /content/human_parse_output/cihp_parsing_maps/* /content/HR-VITON/data/test/image-parse-v3

### Densepose

In [ ]:
!python -m pip install -e detectron2

In [ ]:
%cd detectron2/projects/DensePose
!pip install av>=8.0.3 opencv-python-headless>=4.5.3.56 scipy>=1.5.4

In [28]:
# For getting same input as HR-VITON, change ./densepose/vis/densepose_results.py in line 320
# alpha=0.7 to 1
# inplace=True to False

In [29]:
# change ./densepose/vis/base.py, line 38
# image_target_bgr = image_bgr * 0
# to
# image_target_bgr = image_bgr
# image_target_bgr *= 0

In [30]:
# To save file with name kept and in directory, change apply_net.py, line 286 and 287 to below

# out_fname = './image-densepose/' + image_fpath.split('/')[-1]
# out_dir = './image-densepose'

In [31]:
# !python apply_net.py show configs/densepose_rcnn_R_50_FPN_s1x.yaml \
# https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl \
# /content/HR-VITON/data/test/image dp_segm -v

In [32]:
%cd /content

/content


In [33]:
# !mv /content/image-densepose /content/HR-VITON/data/test/

### Cloth mask

In [ ]:
!pip install carvekit_colab

In [ ]:
from carvekit.ml.files.models_loc import download_all
download_all();

In [36]:
# !mkdir ./cloth
# !cp ./test/cloth/000* ./cloth/

In [37]:
!mkdir /content/HR-VITON/data/test/cloth-mask

In [38]:
# #title Upload images from your computer
# #markdown Description of parameters
# #markdown - `SHOW_FULLSIZE`  - Shows image in full size (may take a long time to load)
# #markdown - `PREPROCESSING_METHOD`  - Preprocessing method
# #markdown - `SEGMENTATION_NETWORK`  - Segmentation network. Use `u2net` for hairs-like objects and `tracer_b7` for objects
# #markdown - `POSTPROCESSING_METHOD`  - Postprocessing method
# #markdown - `SEGMENTATION_MASK_SIZE` - Segmentation mask size. Use 640 for Tracer B7 and 320 for U2Net
# #markdown - `TRIMAP_DILATION`  - The size of the offset radius from the object mask in pixels when forming an unknown area
# #markdown - `TRIMAP_EROSION`  - The number of iterations of erosion that the object's mask will be subjected to before forming an unknown area

# import os
# import numpy as np
# from PIL import Image, ImageOps
# from carvekit.web.schemas.config import MLConfig
# from carvekit.web.utils.init_utils import init_interface

# SHOW_FULLSIZE = False #param {type:"boolean"}
# PREPROCESSING_METHOD = "none" #param ["stub", "none"]
# SEGMENTATION_NETWORK = "tracer_b7" #param ["u2net", "deeplabv3", "basnet", "tracer_b7"]
# POSTPROCESSING_METHOD = "fba" #param ["fba", "none"]
# SEGMENTATION_MASK_SIZE = 640 #param ["640", "320"] {type:"raw", allow-input: true}
# TRIMAP_DILATION = 30 #param {type:"integer"}
# TRIMAP_EROSION = 5 #param {type:"integer"}
# DEVICE = 'cuda' # 'cuda', 'cpu'

# config = MLConfig(segmentation_network=SEGMENTATION_NETWORK,
#                   preprocessing_method=PREPROCESSING_METHOD,
#                   postprocessing_method=POSTPROCESSING_METHOD,
#                   seg_mask_size=SEGMENTATION_MASK_SIZE,
#                   trimap_dilation=TRIMAP_DILATION,
#                   trimap_erosion=TRIMAP_EROSION,
#                   device=DEVICE)

# interface = init_interface(config)

# imgs = []
# root = './HR-VITON/data/test/cloth'
# for name in os.listdir(root):
#     imgs.append(root + '/' + name)

# images = interface(imgs)
# for i, im in enumerate(images):
#     img = np.array(im)
#     img = img[...,:3] # no transparency
#     idx = (img[...,0]==130)&(img[...,1]==130)&(img[...,2]==130) # background 0 or 130, just try it
#     img = np.ones(idx.shape)*255
#     img[idx] = 0
#     im = Image.fromarray(np.uint8(img), 'L')
#     im.save(f'./HR-VITON/data/test/cloth-mask/{imgs[i].split("/")[-1].split(".")[0]}.jpg')

### Parse agnostic

In [40]:
# Here is the parse label and corresponding body parts. You may need or not.
# 0 - 20
# Background
# Hat
# Hair
# Glove
# Sunglasses
# Upper-clothes
# Dress
# Coat
# Socks
# Pants
# tosor-skin
# Scarf
# Skirt
# Face
# Left-arm
# Right-arm
# Left-leg
# Right-leg
# Left-shoe
# Right-shoe

In [41]:
import json
from os import path as osp
import os
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm

def get_im_parse_agnostic(im_parse, pose_data, w=768, h=1024):
    label_array = np.array(im_parse)
    parse_upper = ((label_array == 5).astype(np.float32) +
                    (label_array == 6).astype(np.float32) +
                    (label_array == 7).astype(np.float32))
    parse_neck = (label_array == 10).astype(np.float32)

    r = 10
    agnostic = im_parse.copy()

    # mask arms
    for parse_id, pose_ids in [(14, [2, 5, 6, 7]), (15, [5, 2, 3, 4])]:
        mask_arm = Image.new('L', (w, h), 'black')
        mask_arm_draw = ImageDraw.Draw(mask_arm)
        i_prev = pose_ids[0]
        for i in pose_ids[1:]:
            if (pose_data[i_prev, 0] == 0.0 and pose_data[i_prev, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
                continue
            mask_arm_draw.line([tuple(pose_data[j]) for j in [i_prev, i]], 'white', width=r*10)
            pointx, pointy = pose_data[i]
            radius = r*4 if i == pose_ids[-1] else r*15
            mask_arm_draw.ellipse((pointx-radius, pointy-radius, pointx+radius, pointy+radius), 'white', 'white')
            i_prev = i
        parse_arm = (np.array(mask_arm) / 255) * (label_array == parse_id).astype(np.float32)
        agnostic.paste(0, None, Image.fromarray(np.uint8(parse_arm * 255), 'L'))

    # mask torso & neck
    agnostic.paste(0, None, Image.fromarray(np.uint8(parse_upper * 255), 'L'))
    agnostic.paste(0, None, Image.fromarray(np.uint8(parse_neck * 255), 'L'))

    return agnostic


# if __name__ =="__main__":
#     data_path = './HR-VITON/data/test/'
#     output_path = './HR-VITON/data/test/image-parse-agnostic-v3.2'

#     os.makedirs(output_path, exist_ok=True)

#     for im_name in tqdm(os.listdir(osp.join(data_path, 'image'))):

#         # load pose image
#         pose_name = im_name.replace('.jpg', '_keypoints.json')

#         try:
#             with open(osp.join(data_path, 'openpose_json', pose_name), 'r') as f:
#                 pose_label = json.load(f)
#                 pose_data = pose_label['people'][0]['pose_keypoints_2d']
#                 pose_data = np.array(pose_data)
#                 pose_data = pose_data.reshape((-1, 3))[:, :2]
#         except IndexError:
#             print(pose_name)
#             continue

#         # load parsing image
#         parse_name = im_name.replace('.jpg', '.png')
#         im_parse = Image.open(osp.join(data_path, 'image-parse-v3', parse_name))

#         agnostic = get_im_parse_agnostic(im_parse, pose_data)

#         agnostic.save(osp.join(output_path, parse_name))

### Human agnostic

In [42]:
import json
from os import path as osp
import os
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm

def get_img_agnostic(img, parse, pose_data):
    parse_array = np.array(parse)
    parse_head = ((parse_array == 4).astype(np.float32) +
                    (parse_array == 13).astype(np.float32))
    parse_lower = ((parse_array == 9).astype(np.float32) +
                    (parse_array == 12).astype(np.float32) +
                    (parse_array == 16).astype(np.float32) +
                    (parse_array == 17).astype(np.float32) +
                    (parse_array == 18).astype(np.float32) +
                    (parse_array == 19).astype(np.float32))


    agnostic = img.copy()
    agnostic_draw = ImageDraw.Draw(agnostic)

    length_a = np.linalg.norm(pose_data[5] - pose_data[2])
    length_b = np.linalg.norm(pose_data[12] - pose_data[9])
    point = (pose_data[9] + pose_data[12]) / 2
    pose_data[9] = point + (pose_data[9] - point) / length_b * length_a
    pose_data[12] = point + (pose_data[12] - point) / length_b * length_a
    r = int(length_a / 16) + 1

    # mask arms
    agnostic_draw.line([tuple(pose_data[i]) for i in [2, 5]], 'gray', width=r*10)
    for i in [2, 5]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'gray', 'gray')
    for i in [3, 4, 6, 7]:
        if (pose_data[i - 1, 0] == 0.0 and pose_data[i - 1, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
            continue
        agnostic_draw.line([tuple(pose_data[j]) for j in [i - 1, i]], 'gray', width=r*10)
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'gray', 'gray')

    # mask torso
    for i in [9, 12]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*3, pointy-r*6, pointx+r*3, pointy+r*6), 'gray', 'gray')
    agnostic_draw.line([tuple(pose_data[i]) for i in [2, 9]], 'gray', width=r*6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [5, 12]], 'gray', width=r*6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [9, 12]], 'gray', width=r*12)
    agnostic_draw.polygon([tuple(pose_data[i]) for i in [2, 5, 12, 9]], 'gray', 'gray')

    # mask neck
    pointx, pointy = pose_data[1]
    agnostic_draw.rectangle((pointx-r*7, pointy-r*7, pointx+r*7, pointy+r*7), 'gray', 'gray')
    agnostic.paste(img, None, Image.fromarray(np.uint8(parse_head * 255), 'L'))
    agnostic.paste(img, None, Image.fromarray(np.uint8(parse_lower * 255), 'L'))

    return agnostic

# if __name__ =="__main__":
#     data_path = './HR-VITON/data/test/'
#     output_path = './HR-VITON/data/test/agnostic-v3.2'

#     os.makedirs(output_path, exist_ok=True)

#     for im_name in tqdm(os.listdir(osp.join(data_path, 'image'))):

#         # load pose image
#         pose_name = im_name.replace('.jpg', '_keypoints.json')

#         try:
#             with open(osp.join(data_path, 'openpose_json', pose_name), 'r') as f:
#                 pose_label = json.load(f)
#                 pose_data = pose_label['people'][0]['pose_keypoints_2d']
#                 pose_data = np.array(pose_data)
#                 pose_data = pose_data.reshape((-1, 3))[:, :2]
#         except IndexError:
#             print(pose_name)
#             continue

#         # load parsing image
#         im = Image.open(osp.join(data_path, 'image', im_name))
#         label_name = im_name.replace('.jpg', '.png')
#         im_label = Image.open(osp.join(data_path, 'image-parse-v3', label_name))

#         agnostic = get_img_agnostic(im, im_label, pose_data)

#         agnostic.save(osp.join(output_path, im_name))

## 2. Main

In [43]:
%cd HR-VITON/

/content/HR-VITON


In [ ]:
!pip install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch-lts -c nvidia
!pip install opencv-python torchgeometry Pillow tqdm tensorboardX scikit-image scipy

In [45]:
!pwd

/content/HR-VITON


In [ ]:
# !cp /content/drive/MyDrive/Colab\ Notebooks/hr-viton-weights.zip /content
# hr-viton-weights.zip
!gdown https://drive.google.com/uc?id=1ypB6mNJZSc5varNrXAMOrlTvvXXSMWjk

In [ ]:
!unzip ./hr-viton-weights.zip

In [48]:
!rm /content/HR-VITON/hr-viton-weights.zip

In [49]:
if not os.path.exists("/content/HR-VITON/data"):
  os.mkdir("/content/HR-VITON/data")
if not os.path.exists("/content/HR-VITON/eval_models/weights"):
  os.mkdir("/content/HR-VITON/eval_models/weights")

In [50]:
!mv /content/HR-VITON/v0.1 /content/HR-VITON/eval_models/weights

In [51]:
# !python3 train_generator.py --cuda True --name test -b 4 -j 8 --gpu_ids 0 --tocg_checkpoint ./checkpoints/tocg_final.pth --occlusion

In [ ]:
!pip install flask
!pip install flask-restx
!pip install pyngrok
!pip install flask-ngrok
!pip install flask_cors

In [53]:
%cd /content

/content


In [54]:
!mkdir static/
!mkdir static/output

In [ ]:
from google.colab import userdata
ngrokauthtoken = userdata.get('ngrokauthtoken')
!ngrok authtoken $ngrokauthtoken

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

In [ ]:
import getpass

from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect("22", "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

In [58]:
# !mkdir ~/.config/ngrok
# !mv ~/.ngrok2/ngrok.yml ~/.config/ngrok

In [59]:
# !ssh root@2.tcp.ngrok.io -p16483

In [60]:
import os
import numpy as np
from PIL import Image, ImageOps
from carvekit.web.schemas.config import MLConfig
from carvekit.web.utils.init_utils import init_interface

def run_openpose(dirname):
  os.system(f"mkdir static/{dirname}/test/openpose_img")
  os.system(f"mkdir static/{dirname}/test/openpose_json")
  os.chdir("openpose")
  os.system(f"./build/examples/openpose/openpose.bin --image_dir ../static/{dirname}/test/image --disable_blending \
  --display 0 --write_json ../static/{dirname}/test/openpose_json --write_images ../static/{dirname}/test/openpose_img \
  --num_gpu 1 --num_gpu_start 0")
  os.chdir("..")

def run_human_parse(dirname):
  os.system(f"mkdir static/{dirname}/test/image-parse-v3")
  os.chdir("CIHP_PGN")
  os.system(f"python ./inf_pgn.py --directory ../static/{dirname}/test/image --output ../human_parse_output")
  os.chdir("..")
  os.system(f"cp /content/human_parse_output/cihp_parsing_maps/* /content/static/{dirname}/test/image-parse-v3")

def run_densepose(dirname):
  os.chdir("detectron2/projects/DensePose")
  os.system(f"python apply_net.py show configs/densepose_rcnn_R_50_FPN_s1x.yaml \
  https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl \
  /content/static/{dirname}/test/image dp_segm -v")
  os.chdir("../../..")
  os.system(f"mv /content/image-densepose /content/static/{dirname}/test/")

def run_cloth_mask(dirname):
  os.system(f"mkdir static/{dirname}/test/cloth-mask")
  SHOW_FULLSIZE = False #param {type:"boolean"}
  PREPROCESSING_METHOD = "none" #param ["stub", "none"]
  SEGMENTATION_NETWORK = "tracer_b7" #param ["u2net", "deeplabv3", "basnet", "tracer_b7"]
  POSTPROCESSING_METHOD = "fba" #param ["fba", "none"]
  SEGMENTATION_MASK_SIZE = 640 #param ["640", "320"] {type:"raw", allow-input: true}
  TRIMAP_DILATION = 30 #param {type:"integer"}
  TRIMAP_EROSION = 5 #param {type:"integer"}
  DEVICE = 'cuda' # 'cuda', 'cpu'

  config = MLConfig(segmentation_network=SEGMENTATION_NETWORK,
                    preprocessing_method=PREPROCESSING_METHOD,
                    postprocessing_method=POSTPROCESSING_METHOD,
                    seg_mask_size=SEGMENTATION_MASK_SIZE,
                    trimap_dilation=TRIMAP_DILATION,
                    trimap_erosion=TRIMAP_EROSION,
                    device=DEVICE)

  interface = init_interface(config)

  imgs = []
  root = f'./static/{dirname}/test/cloth'
  for name in os.listdir(root):
    imgs.append(root + '/' + name)

  images = interface(imgs)
  for i, im in enumerate(images):
    img = np.array(im)
    img = img[...,:3] # no transparency
    idx = (img[...,0]==130)&(img[...,1]==130)&(img[...,2]==130) # background 0 or 130, just try it
    img = np.ones(idx.shape)*255
    img[idx] = 0
    im = Image.fromarray(np.uint8(img), 'L')
    im.save(f'./static/{dirname}/test/cloth-mask/{imgs[i].split("/")[-1].split(".")[0]}.jpg')

def run_parse_agnostic(dirname):
    data_path = f'./static/{dirname}/test/'
    output_path = f'./static/{dirname}/test/image-parse-agnostic-v3.2'

    os.makedirs(output_path, exist_ok=True)

    for im_name in tqdm(os.listdir(osp.join(data_path, 'image'))):

        # load pose image
        pose_name = im_name.replace('.jpg', '_keypoints.json')

        try:
            with open(osp.join(data_path, 'openpose_json', pose_name), 'r') as f:
                pose_label = json.load(f)
                pose_data = pose_label['people'][0]['pose_keypoints_2d']
                pose_data = np.array(pose_data)
                pose_data = pose_data.reshape((-1, 3))[:, :2]
        except IndexError:
            print(pose_name)
            continue

        # load parsing image
        parse_name = im_name.replace('.jpg', '.png')
        im_parse = Image.open(osp.join(data_path, 'image-parse-v3', parse_name))

        agnostic = get_im_parse_agnostic(im_parse, pose_data)

        agnostic.save(osp.join(output_path, parse_name))

def run_human_agnostic(dirname):
    data_path = f'./static/{dirname}/test/'
    output_path = f'./static/{dirname}/test/agnostic-v3.2'

    os.makedirs(output_path, exist_ok=True)

    for im_name in tqdm(os.listdir(osp.join(data_path, 'image'))):

        # load pose image
        pose_name = im_name.replace('.jpg', '_keypoints.json')

        try:
            with open(osp.join(data_path, 'openpose_json', pose_name), 'r') as f:
                pose_label = json.load(f)
                pose_data = pose_label['people'][0]['pose_keypoints_2d']
                pose_data = np.array(pose_data)
                pose_data = pose_data.reshape((-1, 3))[:, :2]
        except IndexError:
            print(pose_name)
            continue

        # load parsing image
        im = Image.open(osp.join(data_path, 'image', im_name))
        label_name = im_name.replace('.jpg', '.png')
        im_label = Image.open(osp.join(data_path, 'image-parse-v3', label_name))

        agnostic = get_img_agnostic(im, im_label, pose_data)

        agnostic.save(osp.join(output_path, im_name))

def run_inference(dirname):
  os.chdir("HR-VITON")
  os.system(f"python3 test_generator.py --occlusion --cuda True --test_name inference \
   --tocg_checkpoint ./eval_models/weights/v0.1/mtviton.pth --gpu_ids 0 --gen_checkpoint ./eval_models/weights/v0.1/gen.pth \
   --datasetting unpaired --dataroot ../static/{dirname}/ --data_list test_pairs.txt --output_dir ../static/{dirname}/output")
  os.chdir("..")

In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import uuid

app = Flask("VirtualFitting")
port = "5000"
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False
# run_with_ngrok(app)  # Start ngrok when app is run

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

CORS(app,  supports_credentials=True)

@app.route("/getprod", methods=["POST"])
def getprod():
  cloth = request.files['product']
  clothname = secure_filename(cloth.filename)
  prod_id = str(request.form['prodId'])
  user_id = str(request.form['userId'])
  dirname = prod_id + "_" + user_id
  filename = dirname + ".jpg"

  os.system(f"mkdir ./static/{dirname}")
  os.system(f"mkdir ./static/{dirname}/test")
  os.system(f"mkdir ./static/{dirname}/test/cloth")
  os.system(f"mkdir ./static/{dirname}/test/image")

  clothpath = f"./static/{dirname}/test/cloth/{filename}"

  cloth.save(clothpath)
  resize_cloth = Image.open(clothpath)
  resize_cloth.resize((768, 1024)).save(clothpath)

  return jsonify({
      "dirname": dirname,
      "clothsrc": f"{request.host_url}static/{dirname}/test/cloth/{filename}",
      "hostUrl": request.host_url
  })

@app.route("/showimage", methods=["POST"])
def showimage():  # GET 요청시 리턴 값에 해당 하는 dict를 JSON 형태로 반환
  image = request.files['image']
  dirname = request.form['dirname']
  filename = dirname + ".jpg"

  imagepath = f"./static/{dirname}/test/image/{filename}"

  image.save(imagepath)

  resize_image = Image.open(imagepath)
  resize_image.resize((768, 1024)).save(imagepath)
  resize_image.resize((768, 1024)).save(imagepath)

  os.system(f"touch ./static/{dirname}/test_pairs.txt")
  textfile = open(f"./static/{dirname}/test_pairs.txt", 'w')
  data = f"{dirname}.jpg {dirname}.jpg\n"
  textfile.write(data)
  textfile.close()

  print("가상피팅 시작!")
  run_openpose(dirname)
  run_human_parse(dirname)
  run_densepose(dirname)
  run_cloth_mask(dirname)
  run_parse_agnostic(dirname)
  run_human_agnostic(dirname)
  run_inference(dirname)

  result_name = dirname + "_" + dirname + ".png"
  os.system(f"mv ./static/{dirname}/output/{result_name} ./static/{dirname}/output/{dirname}.png")
  print("가상피팅 완료!")
  return jsonify({
      "resultSrc": f"{request.host_url}static/{dirname}/output/{dirname}.png"
  })

@app.route("/export", methods=["POST"])
def export():
  dirname = request.form["dirname"]
  filepath = f"./static/{dirname}/output/{dirname}.png"

  return send_file(filepath, mimetype="image/png")


@app.route("/")
def index():
  return "Hello, from colab"

app.run()


In [84]:
ngrok.disconnect(public_url)

In [64]:
# !dpkg -s libc6 | grep Arch